In [9]:
import os
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [10]:
df = pd.read_excel('C:/Users/danil/Desktop/hackaton/Складские остатки/Ведомость остатков на 30.06.2022 (сч. 105).xlsx')

In [11]:
df.head(100)


,МОЛ,Цена,Количество,Сумма
0,Место хранения,NaN,NaN,NaN
1,Счет,NaN,NaN,NaN
2,КФО,NaN,NaN,NaN
3,Номенклатура,NaN,NaN,NaN
4,Кузнецов Максим Сергеевич,NaN,4371.0,1613559.99
...,...,...,...,...
95,"Насадка для шуруповерта Torsion PH2 25мм, шт",19.04,16.0,304.64
96,Настенный кондиционер DAIKIN FAA71A/RR71BV Nor...,267641.64,1.0,267641.64
97,Нож кровельный Stenley 99 RETRACTABLE 2-10-099...,452.08,1.0,452.08
98,"Отвертка тестер 70-250 В,SPARTA, шт",259.35,2.0,518.70


In [12]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [14]:
res = df.loc[df['МОЛ'].apply(lambda x: similar(x, 'клей')) > 0.6]

KeyError: 'False: boolean label can not be used without a boolean index'